# Value iteration in practice

#### Frozen lake 환경에서 value iteration 수행하기 

## 주요 데이터 구조 
#### - 보상 테이블(Reward table)
- 각 상태에서 액션을 취해 다음 상태로 갔을 때 얻은 보상에 대한 사전
- {상태, 액션, 다음상태 : 보상} 

#### - 전이 테이블(Transition table)
- 각 상태에서 경험된 다음상태(전이)의 횟수(counter)에 대한 사전
- 전이의 확률을 추정할 때 사용 
- {상태, 액션 : {다음상태 : 횟수}}
- 예)
    - 상태0에서 액션1을 10번 수행했을때, 3번 수행하면 상태4, 7번 수행하면 상태5
    - {(0, 1) : {4 : 3, 5 : 7}}
    
#### - 가치 테이블(Value table)
- {상태 : 상태의 가치}


## 전반적인 로직 
#### 1) 환경에서 100번의 랜덤 스텝을 수행하여 보상테이블, 전이테이블의 값을 채움 
#### 2) 모든 상태에 대해 가치반복을 수행하여 가치테이블 업데이트 
#### 3) 업데이트된 가치테이블의 향상을 확인하기 위해 몇 개의 full 에피소드를 테스트로 수행 
#### 3-1) 만약, 테스트 에피소드들의 평균 보상이 0.8 이상이면 학습 종료 
#### 3-2) 테스트 에피소드를 수행하는 동안 환경으로 부터 얻은 모든 정보를 통해 보상테이블, 전이테이블 업데이트 

In [2]:
import gym
import collections
from tensorboardX import SummaryWriter

In [3]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20

#### - Agent 클래스 
- 학습 루프에서 사용할 테이블들과 함수들을 포함하는 클래스

- __init__() 
    - 환경 생성 
    - 첫번째 관찰 얻음 
    - 필요한 테이블들 생성 (보상, 전이, 가치)

In [7]:
class Agent :
    def __init__(self) :
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset() 
        self.rewards = collections.defaultdict(float) #key값이 없어도 에러 x
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)

- play_n_random_stpes(count) 
    - 환경으로부터의 랜덤 경험들을 수집
    - 보상테이블, 전이테이블 업데이트 
    - 학습을 위해 에피소드가 끝날 때까지 기다릴필요 없음 
        - cross-entropy 방법과의 차이점 

In [8]:
    def play_n_random_steps(self, count):
        
        for _ in range(count):
            action = self.env.action_space.sample() #random experience
            
            new_state, reward, is_done, _ = self.env.step(action)
            
            #reward table 
            #{state, action, new state : reward}
            self.rewards[(self.state, action, new_state)] = reward 
             
            #transition table 
            #{state, action : {new state : count}}
            self.transits[(self.state, action)][new_state] += 1
            
            #is_done=True(에피소드가 종료되면)이면 새로운 관찰을 얻음 
            #에피소드가 종료되지 않았으면 새로운 상태가 현재상태가 됨
            self.state = self.env.reset() if is_done else new_state 

- calc_action_value(state, action) 
    - 전이테이블, 보상테이블, 가치테이블을 통해 상태로부터의 액션가치 계산
    - 목적 
        - 상태에서 수행할 최적의 액션 선택
        - 가치반복을 통해 새로운 상태가치 계산 
        
    - 과정 
        - 전이테이블로부터 주어진 상태, 액션에 대한 전이 횟수 추출
        - 전이 횟수를 모두 더해 상태에서 액션을 수행했을 때의 총 전이 횟수 계산 
        - 총 전이 횟수와 각 상태로의 전이 횟수를 통해 전이확률 계산 
        - 추정한 전이확률을 이용해서 벨만방정식을 통해 액션가치 계산 
           

In [9]:
    def calc_action_value(self, state, action) :
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        
        action_values = 0.0
        
        #transits : {(state, action) : {new state : count}}
        #target_counts : {new state : count}
        for tgt_state, count in target_counts.items() :
            reward = self.rewards[(state, action, tgt_state)]
            action_value += (count/total) * (reward + GAMMA + self.values[tgt_state])
            
        return action_value

- select_action(state) 
    - 주어진 상태에서 최적의 액션 결정 
    - 모든 가능한 액션들에 대한 가치 계산 후 가장 큰 값을 가지는 액션을 선택 
    - 에이전트는 가치추정에 대해 greedy하게 행동

In [10]:
def select_action(self, state):
    best_action, best_value = None, None
    
    for action in range(self.env.action_space.n):
        action_value = self.calc_action_value(state, action)#각 액션에대한 액션가치 계산
        
        if best_value is None or best_value < action_value :
            best_value = action_value
            best_action = action
            
        return best_action

- play_episode(env)
    - select_action()을 사용해서 액션 선택
    - 주어진 환경에 대해 하나의 full 에피소드 수행 
    - 테스트 에피소드 수행에 사용 
    - 하나의 에피소드를 위해 보상을 쌓음 

In [11]:
def play_episode(env) :
    total_reward = 0.0
    
    state = env.reset()
    
    while True :
        action = self.select_action(state) #best action
        
        new_state, reward, is_done, _ = env.step(action)
        
        self.rewards[(state, action, new_state)] = reward
        self.transits[(state, action)][new_state] += 1
        
        total_reward += reward
        
        if is_done : 
            break  #에피소드가 종료되면 새로운 관찰을 받음 
            
        state = new_state
        
    return total_reward 

- value_iteration() 
    - 환경의 모든 상태에서 도달 가능한 모든 상태에 대한 가치 계산 
    - 그 중 가장 큰 값으로 현재 상태가치 업데이트

In [12]:
def value_iteration(self):
    for state in range(self.env.observation_space.n) :
        state_values = [self.calc_action_value(state, action) 
                           for action in range(self.env.action_space.n)]#벨만방정식을 사용해서 가치계산 
        
        self.values[state] = max(state_values)

In [4]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state

    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            action_value += (count / total) * (reward + GAMMA * self.values[tgt_state])
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [self.calc_action_value(state, action)
                            for action in range(self.env.action_space.n)]
            self.values[state] = max(state_values)


####  - main
- 환경, 에이전트, 기록자 생성 
- 100번의 랜덤 스텝
- 모든 상태에 대해 가치반복 
- 가치테이블을 정책으로 해서 테스트 에피소드 수행 
    - 테스트 에피소드의 평균보상이 0.8 이상이면 학습 종료 
- 텐서보드에 기록 (평균보상 추적, 학습루프 중지 조건 체크용)

In [5]:
if __name__ == "__main__" :
    test_env = gym.make(ENV_NAME)
    
    agent = Agent()
    
    writer = SummaryWriter(comment="-v-iteration")
    
    
    iter_no = 0
    best_reward = 0.0
    
    while True :
        iter_no += 1
        
        agent.play_n_random_steps(100)
        
        agent.value_iteration()
        
        reward = 0.0
        
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
            
        reward /= TEST_EPISODES
        
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break
    writer.close()

Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.650
Best reward updated 0.650 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 22 iterations!


## 결과 
#### - 해결까지 12 ~ 100번 반복 
#### - 실행의 80% 정도는 1초 안에 좋은 정책을 찾음 
#### - cross-entropy 방법과 비교하면 매우 향상된 결과 
#### - 성능 향상의 이유 1
- 액션의 확률적 결과, 에피소드의 길이(평균 6~10)
    - cross-entropy, 에피소드 내에서 어떤 스텝이 좋은건지 실수인지 파악을 못함 
    - 가치 반복, 전이확률추론과 가치예측을 통해 각 상태의 개별적 가치와 액션의 확률적 결과를 포함
    
- 그러므로 가치반복이 더 간단하고 환경으로부터 더 적은 데이터를 요구함(space efficiency)

#### - 성능 향상의 이유 2
- 가치반복은 학습을 시작하기위해 full 에피소드가 필요하지 않음 
- 하나의 데이터(example)을 가지고도 가치 업데이트 가능 
- 하지만 frozen lake처럼 복잡한 보상 구조 (골에 가야만 1)을 가진 경우 최소한 하나의 성공적인 에피소드 필요 
    - 8x8 frozen lake 환경의 경우
        - 해결 때 까지 50 ~ 400 반복
        - 대부분의 시간을 첫번째 성공적인 에피소드를 기다리는데 사용 

In [2]:
collections.defaultdict(float)

defaultdict(float, {})

In [3]:
collections.defaultdict(collections.Counter)

defaultdict(collections.Counter, {})